In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import Data_preparation

In [2]:
# First we check if the website festival-alarm.com is reachable
url = "https://www.festival-alarm.com/us"
r = requests.get(url)
print(r)

<Response [200]>


In [3]:
from Data_preparation import DataScraper, TableToDataFrame

urls = [f"https://www.festival-alarm.com/us/Festivals-{year}" for year in range(2014, 2025)]

festivals = [] # Initialize an empty list containing all the festivals from 2014 to 2025

# Iterate over the list of URLs
for url in urls:
    # Get the year from the URL
    year = url.split("-")[-1]
    
    # Scrape data
    table = DataScraper(url)
    
    # Convert data to DataFrame
    final_df = TableToDataFrame(table)
    
    # Rename the "Date {year}" column to "Date"
    final_df.rename(columns={f"Date {year}": "Date"}, inplace=True)

    # Add a new column "year" with the corresponding year value
    final_df['year'] = year

    # Append the DataFrame to the list
    festivals.append(final_df)

    # Dynamically assign variables
    globals()[f"table_{year}"] = table
    globals()[f"final_df_{year}"] = final_df

# Concatenate all final_df DataFrames vertically
festivals_tot = pd.concat(festivals, ignore_index=True)

# Display the combined DataFrame
# print(festivals_tot)

/Users/vincenzosoragnese/Desktop/Data Science/Festival/Data_preparation.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'FullForceFestival' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iat[x,y] = cleaned_data[x][y]
/Users/vincenzosoragnese/Desktop/Data Science/Festival/Data_preparation.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '07/04
-

07/06' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iat[x,y] = cleaned_data[x][y]
/Users/vincenzosoragnese/Desktop/Data Science/Festival/Data_preparation.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3days' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iat[x,y] = cleaned_data[x

In [4]:
festivals_tot.shape

(3613, 11)

In [5]:
festivals_tot.head(10)

,Name,Date,Duration,Where,Category,Genres,Country,Venue,Visitors,Price(s.f.),year
0,FullForceFestival,07/04\n-\n\n07/06,3days,outdoor,Metalfestivals,"Crossover,Germanrock,Hardcore(Metal),Metal,Pun...",Saxony,"FlugplatzRoitzschjora\nLöbnitz,\nDE\n04509",28000,€89.95,2014
1,RockharzFestival,07/10\n-\n\n07/12,3days,outdoor,Metalfestivals,"Germanrock,Gothic,Metal,Punk,Rock",Saxony-Anhalt,"FlugplatzBallenstedt\nBallenstedt,\nDE\n06493",12000,€85.85,2014
2,WackenOpenAir,07/31\n-\n\n08/02,3days,outdoor,Metalfestivals,"Comedy,Germanrock,Folk,Gothic,Hardcore(Metal),...",Schleswig-Holstein,"Wacken,\nDE\n25596",75000,€170.00,2014
3,MetalFrenzyFestival,08/14\n-\n\n08/16,3days,outdoor,Metalfestivals,"Germanrock,Gothic,Hardcore(Metal),Metal,Rock",Saxony-Anhalt,ErlebnisbadGardelegen\nMagdeburgerLandstraße20...,1100,€59.90,2014
4,HighfieldFestival,08/14\n-\n\n08/16,3days,outdoor,Mixedfestivals,"Crossover,Germanrock,Electro,Hardcore(Metal),H...",Saxony,"StörmthalerSee\nGroßpösna,\nDE\n04463",25000,€89.00,2014
5,ElbriotFestival,08/16,1day,outdoor,Metalfestivals,"Hardcore(Metal),Metal,Rock",Hamburg,"HamburgerGroßmarkt\nHamburg,\nDE\n20097",10500,€58.00,2014
6,XRockfest,08/17,1day,outdoor,Metalfestivals,"Hardcore(Metal),Metal",Northrhine-Westphalia,ParkplatzXHerfordDiskothek\nBünderStr.82Herfor...,7000,€38.00,2014
7,BrannOpenAir,06/12\n-\n\n06/13,2days,outdoor,Metalfestivals,"Folk,Metal,Rock",Saxony,"AmFlughafenLeipzig/Halle\nLeipzig,\nDE\n04435",nodata,€49.00,2015
8,MetalFrenzyFestival,06/19\n-\n\n06/21,3days,outdoor,Metalfestivals,"Germanrock,Gothic,Hardcore(Metal),Metal,Rock",Saxony-Anhalt,ErlebnisbadGardelegen\nMagdeburgerLandstraße20...,950,€59.90,2015
9,FullForceFestival,07/03\n-\n\n07/05,3days,outdoor,Metalfestivals,"Crossover,Germanrock,Hardcore(Metal),Metal,Pun...",Saxony,"FlugplatzRoitzschjora\nLöbnitz,\nDE\n04509",25000,€91.95,2015


In [6]:
column_datatypes = festivals_tot.dtypes
print(column_datatypes)

Name           object
Date           object
Duration       object
Where          object
Category       object
Genres         object
Country        object
Venue          object
Visitors       object
Price(s.f.)    object
year           object
dtype: object


## Data cleaning and feature engineering

In [7]:
from Data_preparation import improve_readability
# Improve Name column readability by separating strings before a capital letter

# Apply the function to the Name column
festivals_tot['Name'] = festivals_tot['Name'].apply(improve_readability)


In [8]:
from Data_preparation import rewrite_date

for index, row in festivals_tot.iterrows():
    date = row['Date']  # Assuming 'Date' is the name of the date column
    year = row['year']  # Assuming 'year' is the name of the year column
    rewritten_date = rewrite_date(date, year)
    festivals_tot.at[index, 'Date'] = rewritten_date

# Now we can drop the column year
festivals_tot.drop('year', axis=1, inplace=True)

In [9]:
from Data_preparation import extract_duration

# Apply the function to the Duration column
festivals_tot['Duration'] = festivals_tot['Duration'].apply(extract_duration)

In [10]:
from Data_preparation import remove_festivals

# Apply the function to the Category column
festivals_tot['Category'] = festivals_tot['Category'].apply(remove_festivals)

In [11]:
from Data_preparation import clean_venue

# Apply the 2 useful functions to the Venue column
festivals_tot['Venue'] = festivals_tot['Venue'].apply(clean_venue)
festivals_tot['Venue'] = festivals_tot['Venue'].apply(improve_readability)

festivals_tot['Venue'] = festivals_tot['Venue'].replace("D E", "DE")

In [12]:
from Data_preparation import clean_price

festivals_tot['Price(s.f.)'] = festivals_tot['Price(s.f.)'].apply(clean_price)

In [13]:
festivals_tot.head(10)

,Name,Date,Duration,Where,Category,Genres,Country,Venue,Visitors,Price(s.f.)
0,Full Force Festival,07/04/2014 - 07/06/2014,3,outdoor,Metal,"Crossover,Germanrock,Hardcore(Metal),Metal,Pun...",Saxony,"Flugplatz Roitzschjora Löbnitz, D E 04509",28000,89.95
1,Rockharz Festival,07/10/2014 - 07/12/2014,3,outdoor,Metal,"Germanrock,Gothic,Metal,Punk,Rock",Saxony-Anhalt,"Flugplatz Ballenstedt Ballenstedt, D E 06493",12000,85.85
2,Wacken Open Air,07/31/2014 - 08/02/2014,3,outdoor,Metal,"Comedy,Germanrock,Folk,Gothic,Hardcore(Metal),...",Schleswig-Holstein,"Wacken, D E 25596",75000,170.0
3,Metal Frenzy Festival,08/14/2014 - 08/16/2014,3,outdoor,Metal,"Germanrock,Gothic,Hardcore(Metal),Metal,Rock",Saxony-Anhalt,Erlebnisbad Gardelegen Magdeburger Landstraße...,1100,59.9
4,Highfield Festival,08/14/2014 - 08/16/2014,3,outdoor,Mixed,"Crossover,Germanrock,Electro,Hardcore(Metal),H...",Saxony,"Störmthaler See Großpösna, D E 04463",25000,89.0
5,Elbriot Festival,08/16/2014,1,outdoor,Metal,"Hardcore(Metal),Metal,Rock",Hamburg,"Hamburger Großmarkt Hamburg, D E 20097",10500,58.0
6,X Rockfest,08/17/2014,1,outdoor,Metal,"Hardcore(Metal),Metal",Northrhine-Westphalia,Parkplatz X Herford Diskothek Bünder Str.82 H...,7000,38.0
7,Brann Open Air,06/12/2015 - 06/13/2015,2,outdoor,Metal,"Folk,Metal,Rock",Saxony,"Am Flughafen Leipzig/ Halle Leipzig, D E 04435",nodata,49.0
8,Metal Frenzy Festival,06/19/2015 - 06/21/2015,3,outdoor,Metal,"Germanrock,Gothic,Hardcore(Metal),Metal,Rock",Saxony-Anhalt,Erlebnisbad Gardelegen Magdeburger Landstraße...,950,59.9
9,Full Force Festival,07/03/2015 - 07/05/2015,3,outdoor,Metal,"Crossover,Germanrock,Hardcore(Metal),Metal,Pun...",Saxony,"Flugplatz Roitzschjora Löbnitz, D E 04509",25000,91.95
